In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
import csv

In [ ]:
sc = pyspark.SparkContext(AppName = "Titanic")

In [ ]:
# Cargamos el archivo csv y creamos un RDD con esa información 
# raw: RDD de listas de cadenas de texto
# Primero leemos el arhcivo, dividimos el CSV en listas,
# eliminamos la cabecera

# Solo contamos sobrevivientes
raw = ( sc.textfile("D:\\Universidad\Semestre_8\Big Data\Datasets-20220222\titanic.csv")\
       .map(lambda s: list(csv.reader([s]))[0])\
       .filter(lambda l: l[0] != "survived") )
print("Listas totales: ", raw.count())

# Salida
# Listas totales: 891

In [ ]:
# Funciones
# Se verifica que las listas tengan informacion

def complete(l):
    for i in [0,1,2,3,4,5,6,7]: #Unicamente comprobamos
        if l[i] == '':
            return False
        return True
    
# Transforma la lista de cadenas de texto en diccionarios
# con enteros, reales o cadenas de texto,
# segun la columna.
# no se toman en cuenta las demas columnas:
# class, who, adult_male, deck, embark_town, alive y alone

def proyect_and_parse(l):
    return {"survived": int(l[0]), "pclass": int(l[1]),
            "sex": l[2], "age": float (l[3]),
            "sibsp": int(l[4]),
            "parch": int(l[5]), "fare": float(l[6]),
            "embarked":l[7]}

In [ ]:
# RDD de diccionarios con valores validos
non_null = raw.filtrer(complete)
r1 = (non_null.map(proyect_and_parse))
print("Listas completas:", r1.count())

# Solo renglones completas, de 891 a 712

# Salida:
# Listas completas: 712

In [ ]:
raw.collect()

In [ ]:
# Funcion que acepta dos diccionarios y combina cada clave
# usando la funcion f
# Suponemos que ambos diccionarios tiene los mismas claves

def combine_dicts( f, d1, d2 ):
    res = dict()
    for k in d1.keys():
        res[k] = f(d1[k], d2[k])
    return res

# Dict que determina los valores maximos para cada columna
maximos = r1.reduce ( lambda d1, d2: combine_dicts (max,d1,d2) )
# Dict que determina valores minimos para cada columna
minimos = r1.reduce ( lambda d1, d2: combine_dicts (min,d1,d2) )

print("Maximos:", maximos)
print("Minimos:", minimos)


# Salida
# Maximos: {'survived': 1, 'pclass': 3, 'sex': 'male', 'age': 80.0, 'sibsp': 5, 'parch': 6, 'fare': 512.3292, 'embarked': 'S'}

# Minimos: {'survived': 0, 'pclass': 1, 'sex': 'female', 'age': 0.42, 'sibsp': 0, 'parch': 0, 'fare': 0.0, 'embarked': 'C'}

In [ ]:
# Función que procesa todas las claves en 'keys' de un diccionario y las
# normaliza usando los diccionarios de valores maximos 'maxs' y minimos 'mins'
# El resto de entradas se queda inalterado

def normalized(d, keys, maxs, mins):
    res = dict()
    for (k,v) in d.times():
        if k in keys:
            res[k] = (v - mins[k]) / (maxs[k] - mins[k]) 
        else:
            res[k] = v
    return res

# normalize: realiza una reescala de valores numericos en el rango de 0-1
# Softwarem SW, soft. = Software, Numero, Num #, No. = Numero 
normalized = r1.map(lambda d: normalize(d, ['age','sibsp', 'parch','fare'], maximos, minimos))
print("Primera entrada: \n", r1.first())
print("Primera entrada normalizada:\n", normalized.first(), '\n')

# Salida:
# Primera entrada:
#  {'survived': 0, 'pclass': 3, 'sex': 'male', 'age': 22.0, 'sibsp': 1, 'parch': 0, 'fare': 7.25, 'embarked': 'S'}

# Primera entrada:
#  {'survived': 0, 'pclass': 3, 'sex': 'male', 'age': 0.271173661.., 
#                                         'sibsp': 0.2, 'parch': 0.0, 'fare': 0.014151057....,'embarked': 'S'}

In [ ]:
# Funcion que actualiza el conjunto de valors unicos con el valor con
# el valor que contiene el diccionario en una columna key

def update_set_dict (s, d, key):
    s.add(d[key]) #Spark permite modificar el primer parametro y devolverlo
    return s

# Diccionario de conjuntos de valores unicos para la columna 'sex'
# set(): crea un diccionario vacio, no permite datos repetidos
sex_values = normalized.aggregate(set(), 
                                  lambda s, d: update_set_dict(s,d,'sex'),
                                  lambda s1,s2: s1.union(s2))

# Diccionario de conjuntos de valores unicos para la columna 'embarked'
# set(): crea un diccionario vacio, no permite datos repetidos
embarked_values = normalized.aggregate(set(), 
                                       lambda s, d: update_set_dict(s,d,'embarked'),
                                       lambda s1,s2: s1.union(s2))

print("Valores unicos de la columna sex:", sex_values)
print("Valores unicos de la columna embarked:", embarked_values)

# Salida
# Valores unicos de la columna sex: {'female', 'male'}
# Valores unicos de la columna embarked: {'Q','C','S'}

In [ ]:
# Funcion que acepta un obeto iterable (lista, conjunto, ndarray ...)
# de valores unicos y devuelve un diccionario para
# traducir valor -> numero.
# Los numeros naturales estan en el rango [0, len(it)]

def values_to_mapping(it):
    values = sorted(it)
    size = len(values)
    return { k:v for (k,v) in zip(values, range(size)) } 

# recibe como parametros objetos iterables del mismo tamaño
# resultado : {k:v}
    
# Construye los diccionarios traductores a partir de los conjuntos de valores unicos
sex_mapping = values_to_mapping(sex_values)
embarked_mapping = values_to_mapping(embarked_values)

print("Mapping para la columna Sex:", sex_mapping)
print("Mapping para la columna Embarked:", embarked_mapping)

# Salida
# Valores unicos de la columna sex: {'female': 0, 'male': 1}
# Valores unicos de la columna embarked: {'C': 0,'Q': 1,'S': 2}

In [ ]:
# Función que dado un diccionario y una clave, cambia su valor
# por su traducción según 'mapping'

def dict_translate(d, key, mapping):
    d[key] = mapping[d[key]]
    return d

# Traduce las entradas 'sex' y 'embarked' de los diccionarios
# por sus valores traducidos

titanic = (normalized.map(lambda d: dict_translate(d, 'sex', sex_mapping))
            .map(lambda d: dict_translate(d, 'embarked', embarked_mapping)))

# Entrada preprocesada: una aplicadas varias transformaciones, los datos
# con algun algoritmo de ML
print("Primera entrda preprocesada: ", titanic.first() )

# Primera entrda preprocesada:
# {'survived': 0, 'pclass': 3, 'sex': 1, 'age': 0.2711736617240513, 
#                              'sibsp': 0.2, 'parch': 0, 'fare': 0.014151057562208049, 'embarked': 2}